In [1]:
import boto3

In [2]:
s3 = boto3.client("s3")

In [3]:
response = s3.list_objects_v2(Bucket ="llm-finetune-datset-bucket",Prefix ="datasets/")

In [4]:
response

{'ResponseMetadata': {'RequestId': 'D8NE8ACST0SW2SPF',
  'HostId': 'mz+VWCZFB3416FOpCOgn9gs5nJLNDxqcW7anCDDtPDToYR86XY5pcwu50gOAEQyGkw46sBPdZEk=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'mz+VWCZFB3416FOpCOgn9gs5nJLNDxqcW7anCDDtPDToYR86XY5pcwu50gOAEQyGkw46sBPdZEk=',
   'x-amz-request-id': 'D8NE8ACST0SW2SPF',
   'date': 'Sat, 21 Feb 2026 06:15:28 GMT',
   'x-amz-bucket-region': 'us-east-2',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Contents': [{'Key': 'datasets/',
   'LastModified': datetime.datetime(2026, 2, 20, 8, 42, 23, tzinfo=tzlocal()),
   'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'ChecksumAlgorithm': ['CRC64NVME'],
   'ChecksumType': 'FULL_OBJECT',
   'Size': 0,
   'StorageClass': 'STANDARD'},
  {'Key': 'datasets/pharma_instruction_data.csv',
   'LastModified': datetime.datetime(2026, 2, 20, 10, 39, 34, tzinfo=tzlocal()),
   'ETag': '"7a3bbfb7acae4d8dc8

In [5]:
for obj in response.get("Contents",[]):
    print(obj)

{'Key': 'datasets/', 'LastModified': datetime.datetime(2026, 2, 20, 8, 42, 23, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 0, 'StorageClass': 'STANDARD'}
{'Key': 'datasets/pharma_instruction_data.csv', 'LastModified': datetime.datetime(2026, 2, 20, 10, 39, 34, tzinfo=tzlocal()), 'ETag': '"7a3bbfb7acae4d8dc88dc1a08501d745"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 2469, 'StorageClass': 'STANDARD'}


In [6]:
for obj in response.get("Contents",[]):
    print(obj["Key"])

datasets/
datasets/pharma_instruction_data.csv


In [7]:
dataset_path = "s3://llm-finetune-datset-bucket/datasets/pharma_instruction_data.csv"

In [8]:
from datasets import load_dataset

In [9]:
dataset_path

's3://llm-finetune-datset-bucket/datasets/pharma_instruction_data.csv'

In [11]:
from datasets import load_dataset

dataset = load_dataset(
    "csv",
    data_files={"train": "pharma_instruction_data.csv"}
)

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 5
    })
})

In [13]:
print(dataset["train"])

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5
})
